# Análisis Exploratorio de datos

## Proceso de ETL

### Importación de librerías

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

### Funciones generales


In [13]:

def to_snake_case(text):
    text = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', text)
    text = re.sub('([a-z0-9])([A-Z])', r'\1_\2', text)
    return text.lower().replace(' ', '_')


def convert_int_format(cadena):
    try:
        # Intentar convertir el string a float y luego a entero
        return str(int(float(cadena)))
    except ValueError:
        # Si no es un número, devolver el valor original
        return cadena

### Seteo de rutas

In [14]:
sheet_field_names = 'Variables'
first_row_excels= 16
sheet_dict_values = 'Valores'


path_dem_adults_catalog = '.\Data\DataDemografica\AdolescentesAdultos\ensafrec_ad2022_entrega_w.Catlogo.xlsx'
path_dem_adults_data = '.\Data\DataDemografica\AdolescentesAdultos\ensafrec_ad2022_entrega_w.csv'


path_food_adults_catalog = '.\Data\Alimentos\AdolescentesAdultos\ensafrec_ad2022_rec_w.Catlogo.xlsx'
path_food_adults_data = '.\Data\Alimentos\AdolescentesAdultos\ensafrec_ad2022_rec_w.csv'

### Lectura de datos

In [15]:
## Datos demográficos

df_dem_adults_catalog = pd.read_excel(path_dem_adults_catalog, sheet_name=sheet_field_names, skiprows=first_row_excels)
df_dict_values_dem_adults = pd.read_excel(path_dem_adults_catalog, sheet_name=sheet_dict_values, skiprows=first_row_excels,decimal=',')
df_dem_adults_data_raw = pd.read_csv(path_dem_adults_data, sep=';')

## Datos de alimentación

df_food_adults_catalog = pd.read_excel(path_food_adults_catalog, sheet_name=sheet_field_names, skiprows=first_row_excels)
df_dict_values_food_adults = pd.read_excel(path_food_adults_catalog, sheet_name=sheet_dict_values, skiprows=first_row_excels,decimal=',')
df_food_adults_data_raw = pd.read_csv(path_food_adults_data, sep=';')


### Procesamiento de datos

Funciones de procesamiento de datos

In [16]:
def transform_df_dictionary(df):
    # Asignar nombres a las columnas
    df.columns = ['Valor', 'ID', 'Etiqueta']
    
    
    # Aplicar la función para convertir los valores en la columna 'ID'
    df['ID'] = df['ID'].apply(convert_int_format)
    
    # Rellenar los valores faltantes en la columna 'Valor' con el valor anterior
    df.fillna(method='ffill', inplace=True)
    
    # Agrupar por 'Valor' y crear el diccionario
    df = df.groupby('Valor').apply(lambda x: dict(zip(x['ID'], x['Etiqueta']))).reset_index()
    
    # Asignar los nuevos nombres de columna
    df.columns = ['Campo', 'Diccionario']
    
    return df



def procesar_dataframe(df_data, df_catalog, df_dict):
    
    # Iterar por cada columna en df_data
    for column in df_data.columns:
        # Verificar si la columna está en el catálogo
        if column in df_catalog['Variable'].values:
            # Verificar si la columna está en el DataFrame de diccionarios
            if column in df_dict['Campo'].values:
                print(f"Procesando columna: {column}")
                # Convertir valores de la columna usando convert_int_format
                df_data[column] = df_data[column].apply(convert_int_format)
                # Mapear los valores según el diccionario correspondiente
                df_data[column] = df_data[column].map(df_dict.loc[df_dict['Campo'] == column, 'Diccionario'].values[0])
            
            # Obtener el nuevo nombre de la columna del catálogo
            new_column_name = df_catalog.loc[df_catalog['Variable'] == column, 'Etiqueta'].values[0]
            # Renombrar la columna usando to_snake_case
            df_data.rename(columns={column: to_snake_case(new_column_name)}, inplace=True)
    
    return df_data

Construcción de dataframe de diccionaros para los valores de los datos demográficos

In [17]:
df_dict_values_dem_adults = transform_df_dictionary(df_dict_values_dem_adults)
df_dict_values_dem_adults.head()

,Campo,Diccionario
0,completa,{'1': 'Favor de continuar (Entrevista Completa...
1,consumo,"{'1': 'Igual', '2': 'Mayor', '3': 'Menor'}"
2,estrato,"{'1': 'Rural ( <2500 Hab )', '2': 'Urbano ( 2..."
3,nota1,"{'1': '¿Acepta usted? ,,,, Sí', '2': '¿Acepta ..."
4,nota2,{'1': 'Favor de continuar'}


Ajuste de nombres de columna y valores de los datos demográficos

In [18]:
df_dem_adults_data = procesar_dataframe(df_dem_adults_data_raw, df_dem_adults_catalog, df_dict_values_dem_adults)

Procesando columna: nota1
Procesando columna: region
Procesando columna: sexo
Procesando columna: nota2
Procesando columna: consumo
Procesando columna: completa
Procesando columna: estrato


Construcción de dataframe de de diccionaros para los valores de los datos demográficos

In [8]:
df_dict_values_food_adults = transform_df_dictionary(df_dict_values_food_adults)
df_dict_values_food_adults.head()

,Campo,Diccionario
0,esp_alim,"{'443': 'Leche en polvo entera', '445': 'Leche..."
1,estrato,"{'1': 'Rural ( <2500 Hab )', '2': 'Urbano ( 2..."
2,pa1,"{'0': 'Nunca', '1': '1 día', '2': '2 días', '3..."
3,pa2,"{'1': '1 vez', '2': '2 veces', '3': '3 veces',..."
4,pa3,"{'1': 'POCA', '2': 'MODERADA', '3': 'MUCHA', '..."


In [9]:
df_food_adults_data = procesar_dataframe(df_food_adults_data_raw, df_food_adults_catalog, df_dict_values_food_adults)

Procesando columna: pa1
Procesando columna: esp_alim
Procesando columna: pa2
Procesando columna: pa3
Procesando columna: estrato
Procesando columna: region


In [40]:
df_food_adults_data_pivot = df_food_adults_data.pivot_table(index=['folio', 'folio_de_integrante', 'entidad', 'desc_ent', 'municipio',
       'desc_mun','ponderador', 'estrato_urbanidad/ruralidad',
       '__estrato_de_seleccion', 'unidad_primaria_de_muestreo', 'completa',
       'region'], columns='alimento', values=['¿_cuántos_días_comió_(tomó)_usted?', 'especifique_otra_leche',
       '¿_cuántas_veces_al_día_comió_(tomó)_usted?', 'tamaño_de_porción',
       'número_de_porciones'], aggfunc='first')


In [41]:
df_food_adults_data_pivot = df_food_adults_data_pivot.reset_index()
df_food_adults_data_pivot.columns = [''.join(col).strip() for col in df_food_adults_data_pivot.columns.values]
df_food_adults_data_pivot

,folio,folio_de_integrante,entidad,desc_ent,municipio,desc_mun,ponderador,estrato_urbanidad/ruralidad,__estrato_de_seleccion,unidad_primaria_de_muestreo,...,"¿_cuántos_días_comió_(tomó)_usted?Verduras congeladas como chícharo, zanahoria, brócoli, coliflor, ejotes","¿_cuántos_días_comió_(tomó)_usted?Verduras envasadas como chícharo, zanahoria, champiñones y ejotes.",¿_cuántos_días_comió_(tomó)_usted?Yakult o similares,¿_cuántos_días_comió_(tomó)_usted?Yogur de vaso: a) Entero natural,¿_cuántos_días_comió_(tomó)_usted?Yogur de vaso: b) Entero con frutas,"¿_cuántos_días_comió_(tomó)_usted?Yogur de vaso: c) Bajo en grasa o light natural o con fruta (vitalinea, alpura light, lala light, etc.)",¿_cuántos_días_comió_(tomó)_usted?Yogur para beber: a) Entero natural,¿_cuántos_días_comió_(tomó)_usted?Yogur para beber: b) Entero con fruta,"¿_cuántos_días_comió_(tomó)_usted?Yogur para beber: c) Bajo en grasa o light natural o con fruta (vitalinea, activia 0%, Siluette)",¿_cuántos_días_comió_(tomó)_usted?Zanahoria
0,2022_01001001,2022_01001001_01,1,01 AGUASCALIENTES,1,001 AGUASCALIENTES,"8717,1854790009",Metropolitano (100mil y + Hab),13,0100100011320,...,Nunca,Nunca,Nunca,Nunca,Nunca,Nunca,Nunca,Nunca,Nunca,Nunca
1,2022_01001028,2022_01001028_04,1,01 AGUASCALIENTES,1,001 AGUASCALIENTES,"51475,173555678",Metropolitano (100mil y + Hab),13,0100100011320,...,Nunca,Nunca,Nunca,2 días,Nunca,Nunca,Nunca,Nunca,Nunca,4 días
2,2022_01001034,2022_01001034_03,1,01 AGUASCALIENTES,1,001 AGUASCALIENTES,"16477,8557582843",Metropolitano (100mil y + Hab),13,0100100011320,...,Nunca,Nunca,Nunca,Nunca,1 día,Nunca,1 día,Nunca,Nunca,Nunca
3,2022_01001041,2022_01001041_04,1,01 AGUASCALIENTES,1,001 AGUASCALIENTES,"11801,483709865",Metropolitano (100mil y + Hab),13,0100100011405,...,Nunca,Nunca,Nunca,Nunca,Nunca,Nunca,Nunca,Nunca,Nunca,Nunca
4,2022_01001042,2022_01001042_01,1,01 AGUASCALIENTES,1,001 AGUASCALIENTES,"7032,17435561176",Metropolitano (100mil y + Hab),13,0100100011405,...,Nunca,Nunca,Nunca,Nunca,Nunca,Nunca,Nunca,Nunca,Nunca,1 día
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978,2022_32052044,2022_32052044_02,32,32 ZACATECAS,52,052 VILLA GARCÍA,"17358,8180029514",Rural ( <2500 Hab ),321,3205200000026,...,Nunca,Nunca,Nunca,Nunca,1 día,Nunca,Nunca,5 días,Nunca,1 día
979,2022_32053001,2022_32053001_01,32,32 ZACATECAS,53,053 VILLA GONZÁLEZ ORTEGA,"40367,2050064794","Urbano ( 2500-99,999 Hab)",322,3205300010343,...,Nunca,Nunca,Nunca,Nunca,Nunca,Nunca,Nunca,Nunca,Nunca,5 días
980,2022_32053001,2022_32053001_02,32,32 ZACATECAS,53,053 VILLA GONZÁLEZ ORTEGA,"12447,7763629263","Urbano ( 2500-99,999 Hab)",322,3205300010343,...,Nunca,Nunca,Nunca,Nunca,Nunca,Nunca,4 días,Nunca,Nunca,Nunca
981,2022_32053004,2022_32053004_03,32,32 ZACATECAS,53,053 VILLA GONZÁLEZ ORTEGA,"49050,2282415991","Urbano ( 2500-99,999 Hab)",322,3205300010343,...,Nunca,Nunca,Nunca,Nunca,Nunca,Nunca,Nunca,Nunca,Nunca,1 día


Se crea un df para tener la información de las porciones de los alimentos

In [52]:
df_portions = df_food_adults_data[['identificador_de_alimento', 'alimento', 'porción_estándar']].drop_duplicates()
df_portions.head()

,identificador_de_alimento,alimento,porción_estándar
0,1,Leche Liconsa,1 vaso (240 ml)
1,2,Otra leche opción A,1 vaso (240 ml)
2,3,Otra leche opción B,1 vaso (240 ml)
3,4,Otra leche opción C,1 vaso (240 ml)
4,5,Leche preparada de sabor (chocolate u otro sabor),1 vaso (240 ml)


Pendientes:
- Revisar aleatoriamente si el procesamiento quedó bien


Ideas:
- Crear grupos de alimentos para que no sean tantos